In [25]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import os

### Load Data

In [ ]:
#Read in csv files of 3 datasets 
obesity = pd.read_csv('Data/obesity.csv')
anxiety = pd.read_csv('Data/anxiety_attacks.csv')
panic_attacks = pd.read_csv('Data/panic_attacks.csv')

#Test datasets were imported correctly: remove # in front of data set you want to test
#obesity
#anxiety
#panic_attacks

### Clean the Data

##### Missing Values

In [52]:
#check for missing values
print(obesity.isnull().any().any())
print(anxiety.isnull().any().any())
print(panic_attacks.isnull().any().any())

#drop rows where values are missing
p_dropped_values= panic_attacks.dropna()
print(p_dropped_values.isnull().any().any())

#Drop columns
o_columns_dropped = ['Height', 'weight', 'family_history', 'FAVC', 'FCVC']
#a_columns_to_drop = 
#p_columns_to_drop = 

o_cleaned = obesity.drop([o_columns_dropped], axis=1, inplace=True)
print(o_cleaned.columns)

False
False
True
False


KeyError: "[('Height', 'weight', 'family_history', 'FAVC', 'FCVC')] not found in axis"

In [47]:
obesity.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history', 'FAVC', 'FCVC',
       'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS',
       'Obesity'],
      dtype='object')